In [ ]:
!pip install transformers
!pip install torch
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import re
import random

def delete_uppercase_strings(string_list):
    result_list = []
    for string in string_list:
        string = string.split()
        has_lower = any(any(char.islower() for char in word) for word in string)
        if has_lower:
            string = " ".join(string)
            result_list.append(string)
    return result_list


file_path = "/content/datasets"
dataset = []
for filename in os.listdir(file_path):
    with open(os.path.join(file_path, filename), "r", encoding="utf-8") as infile:
        content = infile.read()
        content = content.split("\n")
        content = delete_uppercase_strings(content)
        content = " ".join(content)
        content = re.split(r"[.]",content)
        updated_content = []
        for sentence in content:
            if len(sentence)!=0:
                updated_content.append(sentence)
        dataset = dataset + updated_content

total_sentences = len(dataset)
train_sentences = int(total_sentences * 0.8)

train_data = dataset[:train_sentences]
eval_data = dataset[train_sentences:]


with open('train.txt', 'w') as file:
    for item in train_data:
        file.writelines(str(item) + '\n')
    

with open('eval.txt', 'w') as file:
    for a in eval_data:
        file.writelines(str(a) + '\n')


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # or specify your own pre-trained model
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load and preprocess text files for fine-tuning
train_file = "train.txt"
eval_file = "eval.txt"

train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_file,
    block_size=128
)
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=eval_file,
    block_size=128
)

# Define the data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output",  # Directory to save the fine-tuned model and training logs
    overwrite_output_dir=True,
    num_train_epochs=8,
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    evaluation_strategy="steps",  # Evaluate every `eval_steps`
    eval_steps=500,
    save_steps=500,
    warmup_steps=100,
    logging_steps=2,
    learning_rate=1e-4,
    fp16=False,
    dataloader_num_workers=4
)

# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine-tuned-model")


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated

Step,Training Loss,Validation Loss
500,3.389300,3.557003
1000,3.309700,3.534105
1500,3.048600,3.531604
2000,2.900700,3.533268
2500,2.950600,3.511909
3000,2.815600,3.555860
3500,2.827600,3.562541
4000,2.649500,3.618344
4500,2.528200,3.626204
5000,2.690600,3.610791


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/util

In [ ]:
# Generate text using the fine-tuned model
generated_text = model.generate(
    input_ids=None,
    max_length=500,  # Adjust the maximum length of the generated text
    num_return_sequences=1,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

# Decode and print the generated text
decoded_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
print(decoded_text)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


” “Kitty, darling Lily,” said Kitty, in her loud, ringing voice with the intensity and grace of a man whose words carry no weight and who knows how to avoid it
 “Mamma won’t be dull either,” said Dolly, in her low voice with the vigor and refinement of an actress
 “Very well, countess, I won’t be dull either 
 I know that!” “All right
” “Kitty, what is it for?” “It for you, because
” “But why are you not worrying about me?” “I have come, haven’t I
 I’ve known him for years, and now you are
” “Well, why _komissant_?” “Oh, good God, do you know what I have known till now
 We!” Kitty’s voice sounded like one of those whispering triumphs over which no man even looks at his face
 “Mamma won’t be dull either
” But as soon as she turned away Kitty was almost crestfallen before the countess, she was too sorry in spite of her sorrow, for Levin
 All her grief came outburntly
 “Yes, and thank heavens, I must, I must
 I love you better
” “I understand now
” “What’s this awful, isn’t it?” “Not to m